In [1]:
import numpy as np 
import pandas as pd
import json
import random
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

# 데이터불러오기

In [ ]:
#!pip install xlrd

In [ ]:
user = pd.read_csv('the-movies-dataset/food.csv')
user.head()

In [ ]:
# #컬럼추출
# meta = meta[['id', 'original_title', 'original_language', 'genres']]
# #컬럼이름 변경
# meta = meta.rename(columns={'id':'movieId'})
# #컬럼 중 일부 데이터만 가져옴(3만2천개)
# meta = meta[meta['original_language'] == 'en']
# meta.head()

In [ ]:
ratings = pd.read_csv('the-movies-dataset/test.csv')

In [ ]:
ratings.head()

# 데이터 전처리

In [ ]:
# num1 = 0
# num2 = 1
# for i in tqdm(test["foodName"]):    
#     if i == test["foodName"][num2]:
#         test["age"][num2] = test["age"][num1]
#         test["gender"][num2] = test["gender"][num1]
#         test["residence"][num2] = test["residence"][num1]
#         test.append(test)
#     num1 += 1
#     num2 += 1
#     if num2 >49999:
#         break

In [ ]:
#test.to_csv('test.csv', index=False, encoding='cp949')

In [ ]:
# num1 = 0
# num2 = 1
# ran = random.randrange(1, 51)
# for i in tqdm(test["foodName"]):
#     if i == test["foodName"][num2]:
#         if test["foodNum"][num1] == test["foodNum"][num2]:
#             test["foodNum"][num2] = ran
#             test.append(test)
#     num1 += 1
#     num2 += 1
#     if num2 >49999:
#         break

In [ ]:
#test.to_csv('test.csv', index=False, encoding='cp949')

# 데이터 가공

In [ ]:
#to_numeric 숫자로 바꿔주는 함수 즉 병합시킬 컬럼을 숫자로 변경시켜주는 것
user.food_num = pd.to_numeric(user.food_num, errors='coerce')
ratings.food_num = pd.to_numeric(ratings.food_num, errors='coerce')

### 이 부분은 생략가능

In [ ]:
# #배열형태로 가공해주기
# def parse_genres(genres_str):
#     #value 값만 가져와서 저장
#     genres = json.loads(genres_str.replace('\'', '"'))
    
#     genres_list = []
#     for g in genres:
#         #키 값이 name이므로 여기 들어 있는 정보를 가져옴
#         genres_list.append(g['name'])

#     return genres_list
# #장르를 함수로 정리된 것으로 변경시켜줌
# meta['genres'] = meta['genres'].apply(parse_genres)

# meta.head()

# 데이터 합병

In [ ]:
#두데이터를 합쳐줌(on = 기준 / how = inner 한줄씩 데이터를 늘리는 걸 말함)
data = pd.merge(ratings, user, on='food_num', how='inner')
#3만2천개
data.head()

# Pivot Table 매소드

In [ ]:
#데이터 생성 / 인덱스 - 유저아이디 / 컬럼 -> 영화제목 / 값 -> 평가
matrix = data.pivot_table(index='food_name', columns='user', values='may').fillna(0)

#컬럼에 영화이름 나열되어 있고 열에는 유저아이디가 수록됨
#671명의 2034개의 컬럼
matrix.head()

In [ ]:
item_user = cosine_similarity(matrix)
item_user = pd.DataFrame(data = item_user, index = matrix.index, columns =matrix.index)
item_user.head()

In [ ]:
def collabor_user(food_name):
    return item_user[food_name].sort_values(ascending=False)[1:6]

In [ ]:
collabor_user("coco")

# 추천시스템 로직

In [7]:
import numpy as np 
import pandas as pd
import json
import random
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

user = pd.read_csv('the-movies-dataset/food.csv')
ratings = pd.read_csv('the-movies-dataset/test.csv')

In [8]:
user.food_num = pd.to_numeric(user.food_num, errors='coerce')
ratings.food_num = pd.to_numeric(ratings.food_num, errors='coerce')

## 아이템 기반 협업 필터링

In [9]:
data = pd.merge(ratings, user, on='food_num', how='inner')

In [10]:
matrix = data.pivot_table(index=['food_name'], columns=["user"], values='may').fillna(0)
item_user = cosine_similarity(matrix)
item_user = pd.DataFrame(data = item_user, index = matrix.index, columns =matrix.index)

def collabor_user(food_name):
    return item_user[food_name].sort_values(ascending=False)[1:6]

In [11]:
collabor_user("sakana")

food_name
kum       0.095559
sinbul    0.094946
zerk      0.092899
zuk       0.092564
sukju     0.091897
Name: sakana, dtype: float64

## 나이 성별 기반 협업 필터링

In [12]:
age = 14
gender = 0
data_age = data[(data['age']==age) & (data['gender']==gender)]

In [17]:
matrix_age = data_age.pivot_table(index=['food_name'], columns=["user"], values='may').fillna(0)
item_age = cosine_similarity(matrix_age)
item_age = pd.DataFrame(data = item_age, index = matrix_age.index, columns =matrix_age.index)

def collabor_age(food_name):
    return item_age[food_name].sort_values(ascending=False)[1:6]

In [18]:
collabor_age("sakana")

food_name
nak     0.369180
hang    0.267359
pong    0.259524
chik    0.253401
han     0.231247
Name: sakana, dtype: float64

## 계절 기반 협업 필터링

In [20]:
#겨울 = 'dec','jan','feb' / 봄 = 'mar','apr','may' / 여름 = 'jun','jul','aug' / 'sep','oct','nov'
season_values = ['mar','apr','may']
season = ['jan','feb','jun','jul','aug','sep','oct','nov','dec']
data_season = data.drop(columns=season,axis = 1)

In [21]:
matrix_season = data_season.pivot_table(index='food_name', columns='user', values = season_values).fillna(0)
item_season = cosine_similarity(matrix_season)
item_season = pd.DataFrame(data = item_season, index = matrix_season.index, columns =matrix_season.index)

def collabor_season(food_name):
    return item_season[food_name].sort_values(ascending=False)[1:6]

In [22]:
collabor_season("sakana")

food_name
kum          0.094506
gwanganri    0.093935
zerk         0.091574
zuk          0.087646
sinbul       0.085942
Name: sakana, dtype: float64